# <h1>Feature extraction subject P03 </h1>

In [58]:
import numpy as np
import pandas as pd
pd.set_option('html',False)

from os import path

In [59]:
CHEST_X = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_X.csv')
CHEST_Y = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_Y.csv')
CHEST_Z = path.relpath('data/P03_DATA_WINDOW/PO3_CHEST_Z.csv')

THIGH_X = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_X.csv')
THIGH_Y = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_Y.csv')
THIGH_Z = path.relpath('data/P03_DATA_WINDOW/PO3_THIGH_Z.csv')

LABEL = path.relpath('data/P03_DATA_WINDOW/P03_LABEL_L.csv')

df_chest_x = pd.read_csv(CHEST_X, header=None, sep='\,')
df_chest_y = pd.read_csv(CHEST_Y, header=None, sep='\,')
df_chest_z = pd.read_csv(CHEST_Z, header=None, sep='\,')

df_thigh_x = pd.read_csv(THIGH_X, header=None, sep='\,')
df_thigh_y = pd.read_csv(THIGH_Y, header=None, sep='\,')
df_thigh_z = pd.read_csv(THIGH_Z, header=None, sep='\,')

df_label = pd.read_csv(LABEL, header=None, sep='\ ')


In [60]:
print len(df_chest_x), len(df_thigh_x)
print min(3,4)


36180 31115
3


<h3>Zero crossing rate</h3>

In [74]:
#Zero crossing
def zero_crossing_rate(l,size):
    return len(np.where(np.diff(np.sign(l)))[0])*1.0 / size

a = [1, 2, 1, 1, -3, -4, 7, 8, 9, 10, -2, 1, -3, 5, 6, 7, -10]
print zero_crossing_rate(a,17)

0.411764705882


<h3>Energy</h3>

In [62]:
def sumOfSqr(axis):
    l = []
    for i in axis:
        l.append(float(i))
    mean = sum(l)/len(l)
    s = 0
    for i in l:
        s += (i-mean)**2
    return s**0.5
    
def extractEnergy(x ,y ,z ,n_sample ,start, size):
    res = []
    for i in range(start, start + size):
        e = sumOfSqr(x.iloc[i]) + sumOfSqr(y.iloc[i]) + sumOfSqr(z.iloc[i])
        e = e/ 3
        e = e / n_sample
        res.append(e)
        
    return pd.DataFrame(np.array(res))

<h2> Function to extract basic information for each window </h2>

In [63]:
'''
http://pandas.pydata.org/pandas-docs/stable/api.html#id5
Information about mean, median... 
'''

def extract(method, data_frame, start, size):
    res = []
    for i in range(start, start + size):
        
        if method == 'mean':
            res.append(data_frame.iloc[i].mean())
        elif method == 'median':
            res.append(data_frame.iloc[i].median())
        elif method == 'max':
            res.append(data_frame.iloc[i].max())
        elif method == 'min':
            res.append(data_frame.iloc[i].min())
        elif method == 'std':
            res.append(data_frame.iloc[i].std())
        elif method == 'energy':
            res.append()
        elif method == 'zero-crossing':
            res.append(zero_crossing_rate(data_frame.iloc[i],size))
        else:
            res.append(data_frame.iloc[i])
        
    return pd.DataFrame(np.array(res))




        

In [64]:


def add_feature(feature_type,df_x,df_y,df_z,path,sensor):
    start=0
    size=min(len(df_chest_x), len(df_thigh_x))-1 
    
    

    FEATURES = path
    try:
        df_features = pd.read_csv(FEATURES, sep='\,')
        
        l = []
        # Get all the featues already computed
        for i in df_features.columns.values:
            n = i.split("_")
            n = n[0] + '_' + n[1]
            l.append(n)
        
        # Create the string format of the feature: feature and sensor
        t = feature_type + "_" + sensor

        
        if t not in l:
            print "Adding feature: ", t
            # This is for all axis
            if feature_type == "energy":
                df_sensor_feature = extractEnergy(df_x,df_y,df_z,len(df_x.iloc[0].values), start, size)
                df_sensor_feature.columns = [feature_type + '_' + sensor ]
                
                df_features= pd.concat([df_features, df_sensor_feature],axis=1)
            # Only for one axis
            else:
                df_x_feature = extract(feature_type,df_x,start,size)
                df_x_feature.columns = [feature_type + '_' + sensor +  '_x' ]
                df_y_feature = extract(feature_type,df_y,start,size)
                df_y_feature.columns = [feature_type + '_' + sensor +  '_y']
                df_z_feature= extract(feature_type,df_z,start,size)
                df_z_feature.columns = [feature_type + '_' + sensor +  '_z' ]
                
                df_features= pd.concat([df_features, df_x_feature,df_y_feature, df_z_feature],axis=1)
            
        
           
        
        
    except IOError:
        print 'file is empty'
        if feature_type == "energy":
            df_sensor_feature = extractEnergy(df_x,df_y,df_z,len(df_x.iloc[0].values), start, size)
            df_sensor_feature.columns = [feature_type + '_' + sensor ]


            df_features= df_sensor_feature
        else:
            df_x_feature = extract(feature_type,df_x,start,size)
            df_x_feature.columns = [feature_type + '_' + sensor +  '_x' ]
            df_y_feature = extract(feature_type,df_y,start,size)
            df_y_feature.columns = [feature_type + '_' + sensor +  '_y']
            df_z_feature= extract(feature_type,df_z,start,size)
            df_z_feature.columns = [feature_type + '_' + sensor +  '_z' ]
            
            df_features= pd.concat([df_x_feature,df_y_feature, df_z_feature],axis=1)



    
    df_features.to_csv(FEATURES, index=False)
    

In [65]:
path = path.relpath('data/FEATURES_P03.csv')
features = ['mean', 'min', 'max', 'median','std', 'energy', 'zero-crossing']


for f in features:
    add_feature(f, df_thigh_x, df_thigh_y, df_thigh_z, path, 'thigh')
    add_feature(f, df_chest_x, df_chest_y, df_chest_z, path, 'chest')

Adding feature:  zero-crossing_thigh
Adding feature:  zero-crossing_chest


<h2>Mean<h2>

In [66]:
start = 0
size = 1000

<h3> Chest </h3>

In [67]:
df_chest_x_mean = extract('std',df_chest_x,start,size)
df_chest_y_mean = extract('std',df_chest_y,start,size)
df_chest_z_mean = extract('std',df_chest_z,start,size)
df_label_ = extract('',df_label,start,size)

df_chest_mean = pd.concat([df_chest_x_mean, df_chest_y_mean, df_chest_z_mean],axis=1,ignore_index=True)
df_chest_mean_with_label = pd.concat([df_chest_mean, df_label_],axis=1,ignore_index=True)


In [68]:
print df_chest_mean_with_label.tail()

            0         1         2   3
995  0.409432  0.424865  0.216726  11
996  0.178615  0.396156  0.250242  11
997  0.169636  0.256009  0.410240  11
998  0.151860  0.222381  0.378154  11
999  0.078437  0.170668  0.434209  11


<h3>Visualize mean values</h3>

In [69]:
import charts

In [70]:
options = dict(
    height=500, 
    title=dict(
        text='Mean chest'),
    chart=dict(zoomType='xy')
)

charts.plot(df_chest_mean_with_label, options=options, show='inline')

<h3>Thigh</h3>

In [71]:
df_thigh_x_mean = extract('std',df_thigh_x,start,size)
df_thigh_y_mean = extract('std',df_thigh_y,start,size)
df_thigh_z_mean = extract('std',df_thigh_z,start,size)
df_label_ = extract('',df_label,start,size)

df_thigh_mean = pd.concat([df_thigh_x_mean, df_thigh_y_mean, df_thigh_z_mean],axis=1,ignore_index=True)
df_thigh_mean_with_label = pd.concat([df_thigh_mean, df_label_],axis=1,ignore_index=True)

In [72]:
options = dict(
    height=500, 
    title=dict(
        text='Mean thigh'),
    chart=dict(zoomType='xy')
)

charts.plot(df_thigh_mean_with_label, options=options, show='inline')

In [73]:

df_chest_mean = df_chest_mean + 5
df_chest_thigh_mean = pd.concat([df_chest_mean,df_thigh_mean], axis=1,ignore_index=True)

options = dict(
    height=500, 
    title=dict(
        text='Mean chest'),
    chart=dict(zoomType='xy')
)

charts.plot(df_chest_thigh_mean, options=options, show='inline')